In [ ]:
import torch
model = torch.hub.load("models/dinov3", 'dinov3_convnext_base', source='local', weights='./checkpoints/dinov3_convnext_small_pretrain_lvd1689m-296db49d.pth')

FileNotFoundError: [Errno 2] No such file or directory: '/home/otter/study/2D-Human-Pose-Estimation/HPE/third_party/dinov3/hubconf.py'